In [23]:
import pandas as pd
from sklearn.pipeline import Pipeline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,FunctionTransformer,PowerTransformer,OneHotEncoder

In [24]:
df=pd.read_csv('updated.csv')
df.head()

,year,month,day,latitude,longitude,zon.winds,mer.winds,humidity,air temp.,s.s.temp.,time_elapsed
0,1980,3,8,-0.02,-109.44,-4.9,1.1,88.293266,25.66,25.97,0
1,1980,3,9,-0.02,-109.44,-4.5,2.2,89.798431,25.69,25.28,1
2,1980,3,10,-0.02,-109.44,-3.8,1.9,89.044724,25.56,24.31,2
3,1980,3,12,-0.02,-109.44,-4.4,0.3,90.482239,24.72,23.64,4
4,1980,3,13,-0.02,-109.44,-3.2,0.1,90.417183,24.66,24.34,5


In [25]:
'''Scaler_pipeline=Pipeline(steps=[
    ('scaler',StandardScaler())
])

log_pipelie=Pipeline(steps=[
    ('log',FunctionTransformer(np.log1p,validate=True)),
    ('scaler',StandardScaler())
])

boxcox_pipeline=Pipeline(steps=[
    ('boxcox',PowerTransformer(method='box-cox')),
    ('scaler',StandardScaler())
])

yeojohnson_pipeline = Pipeline(steps=[
    ('yeojohnson', PowerTransformer(method='yeo-johnson')),
    ('scaler', StandardScaler())
])

preprocessor=ColumnTransformer(transformers=[
    ('yeojohnso',yeojohnson_pipeline,['zon.winds']),
    ('log',log_pipelie,['air temp.']),
    ('scaler',Scaler_pipeline,(df.drop(columns=['zon.winds','air temp.','s.s.temp.'])).columns)
])

preprocessed_data_1=preprocessor.fit_transform(df)'''

"Scaler_pipeline=Pipeline(steps=[\n    ('scaler',StandardScaler())\n])\n\nlog_pipelie=Pipeline(steps=[\n    ('log',FunctionTransformer(np.log1p,validate=True)),\n    ('scaler',StandardScaler())\n])\n\nboxcox_pipeline=Pipeline(steps=[\n    ('boxcox',PowerTransformer(method='box-cox')),\n    ('scaler',StandardScaler())\n])\n\nyeojohnson_pipeline = Pipeline(steps=[\n    ('yeojohnson', PowerTransformer(method='yeo-johnson')),\n    ('scaler', StandardScaler())\n])\n\npreprocessor=ColumnTransformer(transformers=[\n    ('yeojohnso',yeojohnson_pipeline,['zon.winds']),\n    ('log',log_pipelie,['air temp.']),\n    ('scaler',Scaler_pipeline,(df.drop(columns=['zon.winds','air temp.','s.s.temp.'])).columns)\n])\n\npreprocessed_data_1=preprocessor.fit_transform(df)"

In [26]:
#Split the data but not randomly, do it in a sequential order

X_train=df[df['time_elapsed']<=5700].drop(columns=['s.s.temp.'])
X_test=df[df['time_elapsed']>5700].drop(columns=['s.s.temp.'])
y_train=df[df['time_elapsed']<=5700]['s.s.temp.']
y_test=df[df['time_elapsed']>5700]['s.s.temp.']

In [27]:
# Transformations and Scaling

yeojohnson_transformer = PowerTransformer(method='yeo-johnson')
X_train['zon.winds'] = yeojohnson_transformer.fit_transform(X_train[['zon.winds']])
X_test['zon.winds'] = yeojohnson_transformer.transform(X_test[['zon.winds']])

log_transformer = FunctionTransformer(np.log1p, validate=True)
X_train['ait temp.'] = log_transformer.fit_transform(X_train[['air temp.']])
X_test['ait temp.'] = log_transformer.transform(X_test[['air temp.']])

excluded_columns = ['zon.winds', 'air.temp.']
columns_to_scale = [col for col in X_train.columns if col not in excluded_columns]

scaler = StandardScaler()
X_train[columns_to_scale] = scaler.fit_transform(X_train[columns_to_scale])
X_test[columns_to_scale] = scaler.transform(X_test[columns_to_scale])

In [30]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from statsmodels.tsa.arima.model import ARIMA
from sklearn.base import BaseEstimator

# Load your sea surface temperature (SST) data
# Define X_train, X_test, y_train, y_test

# Define custom estimator for ARIMA
class ARIMAWrapper(BaseEstimator):
    def __init__(self, order):
        self.order = order

    def fit(self, X, y):
        self.model = ARIMA(y, order=self.order)
        self.model_fit = self.model.fit()

    def predict(self, X):
        return self.model_fit.forecast(steps=len(X))

# Define parameters for hyperparameter tuning
xgb_param_grid = {'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7], 'n_estimators': [100, 200, 300]}
arima_param_grid = {'order': [(p, d, q) for p in range(6) for d in range(3) for q in range(6)]}

# Perform hyperparameter tuning for XGBoost
xgb_model = XGBRegressor()
xgb_grid_search = GridSearchCV(estimator=xgb_model, param_grid=xgb_param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
xgb_grid_search.fit(X_train, y_train)
xgb_best_params = xgb_grid_search.best_params_

# Perform hyperparameter tuning for ARIMA
arima_model = ARIMAWrapper(order=(1, 1, 1))  # Example order, change as needed
arima_grid_search = GridSearchCV(estimator=arima_model, param_grid=arima_param_grid, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=-1)
arima_grid_search.fit(X_train, y_train)  # Assuming univariate time series
arima_best_params = arima_grid_search.best_params_

# Evaluate models
# Assuming X_test and y_test are available
xgb_best_model = XGBRegressor(**xgb_best_params)
xgb_best_model.fit(X_train, y_train)
y_pred_xgb = xgb_best_model.predict(X_test)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)

best_arima_order = arima_best_params['order']
arima_model = ARIMAWrapper(order=best_arima_order)
arima_model.fit(X_train, y_train)
y_pred_arima = arima_model.predict(X_test)
mse_arima = mean_squared_error(y_test, y_pred_arima)

print("XGBoost MSE:", mse_xgb)
print("ARIMA MSE:", mse_arima)

print("XGBoost Best Parameters:", xgb_best_params)
print("ARIMA Best Parameters:", arima_best_params)


Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 108 candidates, totalling 540 fits


c:\Users\Dell\anaconda3_original\envs\py310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\Dell\anaconda3_original\envs\py310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


XGBoost MSE: 0.31259299753524306
ARIMA MSE: 5.463117895016913
XGBoost Best Parameters: {'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 300}
ARIMA Best Parameters: {'order': (4, 2, 3)}


In [35]:
from sklearn.metrics import r2_score
r_squared_xgb = r2_score(y_test, y_pred_xgb)
r_squared_arima=r2_score(y_test,y_pred_arima)
print("R² Score xgb:", r_squared_xgb)
print("R² Score arima:", r_squared_arima)

R² Score xgb: 0.9402444699031923
R² Score arima: -0.04433403298257321


Our first model